# ResNet50

## 1. Imports:

In [2]:
import pandas as pd
import numpy as np
from numpy import save
from numpy import load
from numpy import asarray
import keras
from sklearn.metrics import confusion_matrix
from keras.layers import Dense, Convolution1D, Flatten, Convolution2D
from tensorflow.keras.layers import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.regularizers import l1
import tensorflow as tf
from keras.utils import to_categorical
from keras.layers import Input, Dense, Dropout, Activation, BatchNormalization, Add
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPool1D, ZeroPadding1D, LSTM, Bidirectional
from keras.models import Sequential, Model
from keras.utils import plot_model
import matplotlib.pyplot as plt
from scipy import optimize
from scipy.io import loadmat
import seaborn as sns
from tensorflow.keras.layers import concatenate

## 2. Data importing:

In [ ]:
#Load the training and test data from 3. Data set generation
#X_train
#y_train
#X_test
#y_test

## 3. ResNet50 structure:

In [3]:
def identity_block(X, f, filters):
    """
    Implements an identity block for ResNet.

    Args:
        X (Tensor): Input tensor to the block.
        f (int): Kernel size for the middle convolutional layer.
        filters (list of int): List containing the number of filters for each convolutional layer (F1, F2, F3).

    Returns:
        Tensor: Output of the identity block.
    """
    F1, F2, F3 = filters  # Unpack filter sizes for each layer

    X_shortcut = X  # Save the input tensor for the shortcut connection

    # First convolutional layer
    X = Conv1D(filters=64, kernel_size=5, activation='relu', strides=1, padding='same')(X)
    X = BatchNormalization()(X)  # Normalize activations

    # Second convolutional layer
    X = Conv1D(filters=64, kernel_size=5, activation='relu', strides=1, padding='same')(X)
    X = BatchNormalization()(X)

    # Third convolutional layer
    X = Conv1D(filters=64, kernel_size=5, activation='relu', strides=1, padding='same')(X)
    X = BatchNormalization()(X)

    # Fourth convolutional layer with `f` kernel size
    X = Conv1D(filters=F2, kernel_size=f, activation='relu', strides=1, padding='same')(X)
    X = BatchNormalization()(X)

    # Fifth convolutional layer with 1x1 kernel
    X = Conv1D(filters=F3, kernel_size=1, activation='relu', strides=1, padding='same')(X)
    X = BatchNormalization()(X)

    # Add the shortcut connection and apply ReLU activation
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X


def convolutional_block(X, f, filters, s=2):
    """
    Implements a convolutional block for ResNet, which includes a downsampling shortcut connection.

    Args:
        X (Tensor): Input tensor to the block.
        f (int): Kernel size for the middle convolutional layer.
        filters (list of int): List containing the number of filters for each convolutional layer (F1, F2, F3).
        s (int): Stride for the first convolutional layer in the block.

    Returns:
        Tensor: Output of the convolutional block.
    """
    F1, F2, F3 = filters  # Unpack filter sizes for each layer

    X_shortcut = X  # Save the input tensor for the shortcut connection

    # First convolutional layer with downsampling
    X = Conv1D(F1, 1, activation='relu', strides=s)(X)
    X = BatchNormalization()(X)

    # Second convolutional layer
    X = Conv1D(F2, f, activation='relu', strides=1, padding='same')(X)
    X = BatchNormalization()(X)

    # Third convolutional layer
    X = Conv1D(F3, 1, strides=1)(X)
    X = BatchNormalization()(X)

    # Shortcut connection with downsampling
    X_shortcut = Conv1D(F3, 1, strides=s)(X_shortcut)
    X_shortcut = BatchNormalization()(X_shortcut)

    # Add the shortcut connection and apply ReLU activation
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X


def ResNet50(input_shape):
    """
    Builds the ResNet50 architecture.

    Args:
        input_shape (tuple): Shape of the input tensor (time steps, features).

    Returns:
        keras.Model: A Keras model instance representing the ResNet50 architecture.
    """
    X_input = Input(input_shape)  # Define input tensor

    # Initial convolutional layer and max pooling
    X = ZeroPadding1D(3)(X_input)
    X = Conv1D(64, 7, strides=2)(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    X = MaxPool1D(pool_size=2, strides=2, padding='same')(X)

    # First convolutional block and identity blocks
    X = convolutional_block(X, f=3, filters=[64, 64, 256], s=1)
    X = identity_block(X, 3, [64, 64, 256])
    X = identity_block(X, 3, [64, 64, 256])

    # Second convolutional block and identity blocks
    X = convolutional_block(X, f=3, filters=[128, 128, 512], s=2)
    X = identity_block(X, 3, [128, 128, 512])
    X = identity_block(X, 3, [128, 128, 512])
    X = identity_block(X, 3, [128, 128, 512])

    # Third convolutional block and identity blocks
    X = convolutional_block(X, f=3, filters=[256, 256, 1024], s=2)
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])

    # Fourth convolutional block and identity blocks
    X = convolutional_block(X, f=3, filters=[512, 512, 2048], s=2)
    X = identity_block(X, 3, [512, 512, 2048])
    X = identity_block(X, 3, [512, 512, 2048])
    X = identity_block(X, 3, [512, 512, 2048])

    # Final pooling and output layer
    X = MaxPool1D(pool_size=2, strides=2, padding='same')(X)
    X = GlobalAveragePooling1D()(X)
    X = Dense(2, activation='sigmoid')(X)  # Output layer with 2 units for binary classification

    model = Model(inputs=X_input, outputs=X, name='ResNet50')

    return model


In [4]:
# Instantiate the ResNet50 model with the specified input shape
# The model is designed for 1D inputs with a sequence length of 5000 and 12 features per time step.
model_new = ResNet50(input_shape=(5000, 12))

# Compile the model with the following configurations:
# - Loss function: Binary Crossentropy for binary classification tasks.
# - Optimizer: Adam optimizer with a learning rate of 0.01 for gradient-based optimization.
# - Metrics: 
#   - BinaryAccuracy: Calculates accuracy based on a threshold (default 0.5).
#   - Recall: Measures the proportion of correctly predicted positive samples.
#   - Precision: Measures the proportion of true positives among all predicted positives.
model_new.compile(
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    metrics=[
        tf.keras.metrics.BinaryAccuracy(name='accuracy', threshold=0.5),
        tf.keras.metrics.Recall(name='Recall'),
        tf.keras.metrics.Precision(name='Precision')
    ]
)


In [ ]:
# Training the model
batchsize = 20 
model_new.fit(x=X_train, y=y_train, validation_data=(X_test,Y_test),epochs=150)